In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()

True

In [9]:
import os
print(os.getenv("OPENROUTER_API_KEY"))

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


In [11]:
model = ChatOpenAI(base_url="https://openrouter.ai/api/v1",model="openai/gpt-oss-120b",api_key=os.getenv("OPENROUTER_API_KEY"))

In [12]:
model.invoke("hi")

AIMessage(content='Hello! 👋 How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 55, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 38, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'cost': 1.2595e-05, 'is_byok': False, 'cost_details': {'upstream_inference_cost': 1.2595e-05, 'upstream_inference_prompt_cost': 2.145e-06, 'upstream_inference_completions_cost': 1.045e-05}}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': None, 'id': 'gen-1770296395-lyVUu0ZqIlEwYNtgYwAD', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2de3-46db-7742-9a56-c7ef72a8d1e2-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 55, 'output_tokens': 55, 'total_tokens': 110, 'input_token_details': {'audio': 0, 'cache_read': 0

# Email fetching part

In [ ]:
from typing import List
from pydantic import BaseModel
from src.ingestion.gmail.gmail_auth import get_gmail_service
from datetime import datetime, timedelta
import base64
import os
from bs4 import BeautifulSoup
import re
ATTACHMENT_DIR = "attachments"
os.makedirs(ATTACHMENT_DIR, exist_ok=True)


# =====================
# SCHEMA
# =====================
class Metadata(BaseModel):
    sender: str
    to: List[str]
    subject: str
    date: str
    labels: List[str]


class Intent(BaseModel):
    email_type: str
    source: str = "heuristic"
    confidence: float | None = None


class Content(BaseModel):
    cleaned_text: str


class EmailObject(BaseModel):
    message_id: str
    thread_id: str
    metadata: Metadata
    intent: Intent
    content: Content
    attachments: List[dict] = []


# =====================
# HELPERS
# =====================
def decode_base64(data):
    return base64.urlsafe_b64decode(data).decode("utf-8", errors="ignore")



def normalize_text(text: str) -> str:
    text = re.sub(r'[\u200B-\u200D\uFEFF]', '', text)


    text = text.replace('\xa0', ' ')

    text = re.sub(r'\s+', ' ', text)

    return text.strip()



def parse_to_header(to_header: str | None) -> List[str]:
    if not to_header:
        return []
    return [addr.strip() for addr in to_header.split(",")]


def clean_html(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")

    for tag in soup(["style", "script", "head", "meta", "noscript"]):
        tag.decompose()

    for img in soup.find_all("img"):
        img.decompose()

    text = soup.get_text(separator="\n")
    lines = [l.strip() for l in text.splitlines() if 40 < len(l.strip()) < 300]

    return "\n".join(lines[:15])


def classify_email(headers, plain, html, labels):
    if "CATEGORY_PROMOTIONS" in labels:
        return "newsletter"
    if "CATEGORY_SOCIAL" in labels:
        return "notification"
    if html and len(html) > 5000:
        return "newsletter"
    if plain and len(plain) < 300:
        return "transactional"
    return "personal"


# =====================
# RAW BODY EXTRACTION
# =====================
def extract_raw_bodies(service, msg_id, payload):
    plain_text = None
    html_text = None
    attachments = []

    def walk(parts):
        nonlocal plain_text, html_text

        for part in parts:
            mime = part.get("mimeType")
            filename = part.get("filename")

            if filename and part["body"].get("attachmentId"):
                att_id = part["body"]["attachmentId"]
                att = service.users().messages().attachments().get(
                    userId="me", messageId=msg_id, id=att_id
                ).execute()

                data = base64.urlsafe_b64decode(att["data"])
                path = os.path.join(ATTACHMENT_DIR, filename)

                with open(path, "wb") as f:
                    f.write(data)

                attachments.append({"filename": filename, "path": path})

            elif part["body"].get("data"):
                data = decode_base64(part["body"]["data"])
                if mime == "text/plain":
                    plain_text = data
                elif mime == "text/html":
                    html_text = data

            if part.get("parts"):
                walk(part["parts"])

    if payload.get("parts"):
        walk(payload["parts"])
    elif payload.get("body", {}).get("data"):
        mime = payload.get("mimeType")
        data = decode_base64(payload["body"]["data"])
        if mime == "text/plain":
            plain_text = data
        elif mime == "text/html":
            html_text = data

    return plain_text, html_text, attachments


# =====================
# MAIN INGESTION
# =====================
def read_emails_in_date_range(start_date: str, end_date: str) -> List[EmailObject]:
    service = get_gmail_service()

    start = datetime.strptime(start_date, "%Y-%m-%d") - timedelta(days=1)
    end = datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)
    query = f"after:{start.strftime('%Y/%m/%d')} before:{end.strftime('%Y/%m/%d')}"

    results = []
    response = service.users().messages().list(
        userId="me", q=query, maxResults=100
    ).execute()

    for m in response.get("messages", []):
        msg = service.users().messages().get(
            userId="me", id=m["id"], format="full"
        ).execute()

        headers = {h["name"]: h["value"] for h in msg["payload"]["headers"]}
        labels = msg.get("labelIds", [])

        plain, html, attachments = extract_raw_bodies(
            service, m["id"], msg["payload"]
        )

        email_type = classify_email(headers, plain, html, labels)

        if email_type == "newsletter" and html:
            cleaned = clean_html(html)
            cleaned = normalize_text(cleaned)
        else:
            cleaned = plain or clean_html(html or "")
            cleaned = normalize_text(plain)

        email = EmailObject(
            message_id=msg["id"],
            thread_id=msg["threadId"],
            metadata=Metadata(
                sender=headers.get("From", ""),
                to=parse_to_header(headers.get("To")),
                subject=headers.get("Subject", ""),
                date=headers.get("Date", ""),
                labels=labels
            ),
            intent=Intent(
                email_type=email_type,
                source="heuristic",
                confidence=0.6
            ),
            content=Content(
                cleaned_text=cleaned
            ),
            attachments=attachments
        )

        results.append(email)

    return results


In [5]:
read_emails_in_date_range("2026-02-1","2026-02-5")

[EmailObject(message_id='19c2f9c404033f88', thread_id='19c2f9c404033f88', metadata=Metadata(sender='Kaggle <no-reply@kaggle.com>', to=['gpt1410@gmail.com'], subject='🏆 Game Arena Winners: Poker, Werewolf, and Chess ', date='Thu, 05 Feb 2026 13:01:34 -0800', labels=['UNREAD', 'CATEGORY_UPDATES', 'INBOX']), intent=Intent(email_type='newsletter', source='heuristic', confidence=0.6), content=Content(cleaned_text="event of 2026 has officially concluded. From the logic of Chess to the social dynamics of Werewolf and the calculated risks of Poker, we’ve seen AI models navigate complex environments in dynamic ways. Here are the champions from the final leaderboards 👇 From each leaderboard you can find all of the underlying datasets, game replays, model reasoning outputs and environments, open sourced for further analysis. Missed the action? We’ve compiled all the expert commentary from Hikaru Nakamura, Nick Schulman, Doug Polk, and Liv Boeree into a single Kaggle, Inc 1600 Amphitheatre Pkwy Mo

In [ ]:
# getting the content now convert the content to proper format

In [20]:
b = [{"m":{"name":"Manas"}},{"m":{"name":"Madnas"}},{"n":{"name":"qwqdqw"}}]
from collections import defaultdict

a = defaultdict(list)

for c in b:
    key = list(c.keys())[0]
    a[key].append(c[key]["name"])

a = dict(a)

In [21]:
print(a)

{'m': ['Manas', 'Madnas'], 'n': ['qwqdqw']}
